# BrainAge Project

In [ ]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.model_selection import KFold
from sklearn.svm import SVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt

In [ ]:
# Parameters
toy = False
path = '/home/ubuntu/fmriprep/'
output_dir = '../output/'

n_jobs = 20 #amount of cores
cv=4

In [ ]:
# Load subject ids
# files = os.listdir(path)
# subj = [n[4:9] for n in files if n.startswith('sub')] #here we will filter the file names to leave only subj number as bellow

input_dir = '/home/ubuntu/'
file = 'subjlist_wfiles_CNP.txt'
with open(input_dir+file, 'r') as f:
    lines = f.readlines()
    
subj = [n.strip()[4:] for n in lines]

random_subj = subj[:]
random.shuffle(random_subj)

In [ ]:
# if toy:
#     subj = subj[:8]
random_subj


In [ ]:
# Load ages
df = pd.read_csv('participants.tsv', sep='\t')

# take age from subj id
y_age = []
for i in random_subj:
    a = df.loc[df['participant_id'] == 'sub-'+i] #whole row
    y_age.append(int(a.age))


# # Load IQ
# df = pd.read_csv('add file here', sep='\t')

# # take age from subj id
# y_age = []
# for i in subj:
#     a = df.loc[df['participant_id'] == 'sub-'+i] #whole row
#     y_iq.append(int(a.id))


In [ ]:
# # Load volumes for each subject into a dictionary
# d = {}
# for i in range(len(subj)):
#     img = nib.load(path+'sub-'+subj[i]+'_T1w_space_MNI152NLin2009cAsym_preproc.nii.gz').get_fdata()
#     d[subj[i]] = img

filenames = []
# data = []
for i in range(len(random_subj)):
#     img = nib.load(path+'sub-'+subj[i]+/anat/'sub-'+subj[i]+'_T1w_space_MNI152NLin2009cAsym_preproc.nii.gz').get_fdata()
#     data.append(img)
    filenames.append(path+'sub-'+random_subj[i]+'/anat/sub-'+random_subj[i]+'_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz')

# data  = np.array(data) 

## Let's see the size of a single volume
# data.shape

# # Flatten 3D volume and append to a list (or feed 3D volume to nilearn)
# X = []

# for i in d.values():
#     flattened = i.flatten()
#     X.append(flattened)
    
# # Turn list into array 
# X = np.array(X)

In [ ]:
filenames

In [ ]:
# Affine resample

# from nilearn.image import resample_to_img
# # img = data.affine

# resampled_imgs = []
# resampled_imgs.append(nib.load(path+filenames[0]))
                      
# for i in range(1,len(filenames)):
#     resampled_img = resample_to_img(path+filenames[i], path+filenames[0])
#     resampled_imgs.append(resampled_img)

# for i in resampled_imgs:
#     print(i.affine)

In [ ]:
# path_and_filenames = [path+n for n in filenames]

nifti_masker = NiftiMasker(
    standardize=False,
    smoothing_fwhm=2, mask_strategy='epi',
    memory='nilearn_cache')  # cache options
gm_maps_masked = nifti_masker.fit_transform(filenames)
n_samples, n_features = gm_maps_masked.shape
print("%d samples, %d features" % (n_samples, n_features))

In [ ]:
print("%d samples, %d features" % (n_samples, n_features))

In [ ]:
%matplotlib inline
from nilearn import plotting
plotting.plot_roi(nifti_masker.mask_img_)
plt.savefig(output_dir+'nilearn_mask')

In [ ]:
X_train = gm_maps_masked[:84]
X_test = gm_maps_masked[84:]
y_train = y_age[:84]
y_test = y_age[84:]
random_subj_train = random_subj[:84]
random_subj_test = random_subj[84:]

# np.savez_compressed(output_dir+'train_set',a=X_train, b=y_train, c=random_subj_train)
# np.savez_compressed(output_dir+'test_set',a=X_test, b=y_test, c=random_subj_test)




In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(gm_maps_masked, y_age, test_size=0.31, random_state=42)

In [ ]:
print(y_train)
print(y_test)

## Gridsearch 
### Models
* Linear Support vector regressor
    * C = 0.1, 1., 10.

### Feature selection
* ANOVA
    * 2000 features
    * 50000 features
    * Use all features (176*256*256=11534336)

In [ ]:
from sklearn.pipeline import Pipeline
toy = False
if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]


# Remove features with too low between-subject variance
# variance_threshold = 

# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# feature_selection = SelectKBest(f_regression)

# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
               

anova_svr = Pipeline([
            ('variance_threshold', VarianceThreshold(threshold=.01)),
            ('anova', SelectKBest(f_regression)),
            ('svr', SVR())])

parameters = [{'anova__k': [100000, 'all'],
               'svr__C': [0.01, 0.1, 1, 10, 100]}]

# parameters = [{'anova__k': [2000],
#                'svr__C': [0.1,1]}]

grid = GridSearchCV(anova_svr, cv=cv, n_jobs=n_jobs, param_grid=parameters, scoring='neg_mean_absolute_error')
grid.fit(X_train, y_train)

# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






In [ ]:
X_train.shape

In [ ]:
print('best score: ',grid.best_score_)
print('best params: ',grid.best_params_)
print('\n\nfullresults: \n', grid.cv_results_)
df1 = pd.DataFrame(grid.cv_results_)
df1

In [ ]:
age_pred = grid.predict(gm_maps_masked)
mean_abs_error = np.mean(np.abs(age_pred-y_age))
df1 = pd.DataFrame(grid.cv_results_)

with open(output_dir+'log.txt', '+a') as f:
    f.write('best score: '+str(grid.best_score_))
    f.write('best params: '+str(grid.best_params_))
    f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
    f.write(str(df1)+'\n')
    f.write(str(age_pred)+'\n')
    f.write(str(y_train)+'\n')
    f.write(str(mean_abs_error)+'\n')
    

In [ ]:
# age_pred = grid.predict(gm_maps_masked)
# np.mean(np.abs(age_pred-y_age))
# y_age

In [ ]:
# r2 score